In [12]:
import pandas as pd
import numpy as np
import pickle

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

0 - Crate <br>
1 - Feeder <br>
2 - Cup <br>
3 - None <br>

In [13]:
model = BayesianNetwork([
        ('CrateAvailable', 'NextObject'),
        ('FeederAvailable', 'NextObject'),
        ('CupAvailable', 'NextObject'),

        ('Worker', 'NextObject')
    ])

In [14]:
data = pd.read_csv(r'data\data.csv')

model.fit(data, estimator=MaximumLikelihoodEstimator)
assert model.check_model()

In [15]:
# cpd_second_last_obj = TabularCPD(
#     variable='SecondLastObject', variable_card=3,
#     values=[[0.33],  # Probability for SecondLastObject being state 0 (e.g., 'Crate')
#             [0.33],  # Probability for SecondLastObject being state 1 (e.g., 'Feeder')
#             [0.34]]  # Probability for SecondLastObject being state 2 (e.g., 'Cup')
# )

# cpd_last_obj = TabularCPD(
#     variable='LastObject', variable_card=3,
#     values=[[0.3, 0.3, 0.4],  # Probabilities for picking Crate next
#             [0.4, 0.3, 0.3],  # Probabilities for picking Feeder next
#             [0.3, 0.4, 0.3]], # Probabilities for picking Cup next
#     evidence=['SecondLastObject'], 
#     evidence_card=[3])

# cpd_cup_available = TabularCPD(variable='CupAvailable', variable_card=2, values=[[0.5], [0.5]])
# cpd_crate_available = TabularCPD(variable='CrateAvailable', variable_card=2, values=[[0.5], [0.5]])
# cpd_feeder_available = TabularCPD(variable='FeederAvailable', variable_card=2, values=[[0.5], [0.5]])

# cpd_worker = TabularCPD(variable='Worker', variable_card=3, values=[[1/3], [1/3], [1/3]])


# cpd_next_obj = TabularCPD(
#     variable='NextObject', variable_card=4,
#     values=[
#         # Probabilities for NextObject being Crate (0)
#         [0.9, 0, 0.6, 0, 0.8, 0, 0.7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        
#         # Probabilities for NextObject being Feeder (1)
#         [0.1, 0, 0.3, 0, 0.1, 0, 0.2, 0, 0.8, 0, 0.6, 0, 0.9, 0, 0.7, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        
#         # Probabilities for NextObject being Cup (2)
#         [0, 0, 0.1, 0, 0.1, 0, 0.1, 0, 0.2, 0, 0.4, 0, 0.1, 0, 0.3, 0, 0.9, 0, 0.8, 0, 1.0, 0, 0.9, 0],

#         # Probabilities for NextObject being None (3)
#         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0.1, 1, 0.2, 1, 0, 1, 0.1, 1]
#     ],
#     evidence=['Worker', 'CrateAvailable', 'FeederAvailable', 'CupAvailable'],
#     evidence_card=[3, 2, 2, 2])

# model.add_cpds(cpd_cup_available, cpd_crate_available, cpd_feeder_available, cpd_next_obj, cpd_worker)
# assert model.check_model()

In [16]:
with open('models/recent.pkl', 'wb') as file:
    pickle.dump(model, file)

example query:

In [17]:
inference = VariableElimination(model)
result = inference.query(
    variables=['NextObject'],
    evidence={
        'CrateAvailable': True,
        'FeederAvailable': True,
        'CupAvailable': True,

        'Worker': 1
        }
)

print(result)

+---------------+-------------------+
| NextObject    |   phi(NextObject) |
+===============+===================+
| NextObject(0) |            0.2000 |
+---------------+-------------------+
| NextObject(1) |            0.6000 |
+---------------+-------------------+
| NextObject(2) |            0.2000 |
+---------------+-------------------+
| NextObject(3) |            0.0000 |
+---------------+-------------------+
